In [1]:
from UNet import Autoencoder
from train_data_down import loader
import torch
import torch.nn as nn
from tqdm import tqdm    # 进度条
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 初始化
model = Autoencoder().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [3]:
epoches = 100
for epoch in range(epoches):
    model.train()
    total_loss = 0
    progress = tqdm(loader, desc=f'Epoch {epoch + 1}/{epoches}')
    for batch in progress:
        x = batch.to(device)
        optimizer.zero_grad()
        # 动态调整掩码比例 (20% -> 60%)
        curr_mask_ratio = 0.2 + 0.4 * (epoch / 100)
        outputs, loss  = model(x)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch} | Loss: {total_loss/len(loader):.4f}  ")

Epoch 1/100:   0%|          | 0/7200 [00:00<?, ?it/s]F:\python\CHB-3\Unet\UNet.py:112: UserWarning: Using a target size (torch.Size([8, 22, 64])) that is different to the input size (torch.Size([8, 22, 512])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(x, decoded)
Epoch 1/100:   0%|          | 0/7200 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (512) must match the size of tensor b (64) at non-singleton dimension 2